In [12]:
import tensorflow as tf
from tensorflow import keras
import numpy as np
from keras.models import Sequential
from keras.layers import Conv2D

In [13]:
fashion_mnist = keras.datasets.fashion_mnist

In [14]:
(train_data, train_label), (test_data, test_label) = fashion_mnist.load_data()

4422102/4422102 [==============================] - 0s 0us/step


In [15]:
train_data.shape

(60000, 28, 28)

In [16]:
train_data = train_data/255.0
test_data = test_data/255.0

In [17]:
train_data = train_data.reshape(len(train_data), 28, 28, 1)
test_data = test_data.reshape(len(test_data), 28, 28, 1)

In [18]:
train_data.shape

(60000, 28, 28, 1)

In [35]:
def build_model(hp):  
  model = keras.Sequential([
    keras.layers.Conv2D(
        filters=hp.Int('conv_1_filter', min_value=32, max_value=128, step=16),
        kernel_size=hp.Choice('conv_1_kernel', values = [3,5]),
        activation='relu',
        input_shape=(28,28,1)
    ),
    keras.layers.Conv2D(
        filters=hp.Int('conv_2_filter', min_value=32, max_value=64, step=16),
        kernel_size=hp.Choice('conv_2_kernel', values = [3,5]),
        activation='relu'
    ),
    keras.layers.Flatten(),
    keras.layers.Dense(
        units=hp.Int('dense_1_units', min_value=32, max_value=128, step=16),
        activation='relu'
    ),
    keras.layers.Dense(10, activation='softmax')
  ])
  
  model.compile(optimizer=keras.optimizers.Adam(hp.Choice('learning_rate', values=[1e-2, 1e-3])),
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])
  
  return model

In [36]:
from keras_tuner import RandomSearch
from keras_tuner.engine.hyperparameters import HyperParameters
# !pip install -q -U keras-tuner

In [37]:
tuner_search = RandomSearch(
          build_model, objective = "val_accuracy", max_trials = 5, directory = "output", project_name = "Fashion Mnist"
          )

In [38]:
tuner_search.search(train_data, train_label, epochs = 3, validation_split = 0.1)

Trial 5 Complete [00h 00m 26s]
val_accuracy: 0.9068333506584167

Best val_accuracy So Far: 0.9151666760444641
Total elapsed time: 00h 03m 05s


In [41]:
model = tuner_search.get_best_models(num_models = 1)[0]

In [42]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 26, 26, 48)        480       
                                                                 
 conv2d_1 (Conv2D)           (None, 24, 24, 32)        13856     
                                                                 
 flatten (Flatten)           (None, 18432)             0         
                                                                 
 dense (Dense)               (None, 112)               2064496   
                                                                 
 dense_1 (Dense)             (None, 10)                1130      
                                                                 
Total params: 2,079,962
Trainable params: 2,079,962
Non-trainable params: 0
_________________________________________________________________


In [43]:
model.fit(train_data, train_label, epochs = 10, validation_split = 0.1, initial_epoch = 3)

Epoch 4/10
1688/1688 [==============================] - 8s 5ms/step - loss: 0.1338 - accuracy: 0.9496 - val_loss: 0.2503 - val_accuracy: 0.9132
Epoch 5/10
1688/1688 [==============================] - 7s 4ms/step - loss: 0.0917 - accuracy: 0.9671 - val_loss: 0.2977 - val_accuracy: 0.9172
Epoch 6/10
1688/1688 [==============================] - 8s 4ms/step - loss: 0.0632 - accuracy: 0.9763 - val_loss: 0.3352 - val_accuracy: 0.9153
Epoch 7/10
1688/1688 [==============================] - 7s 4ms/step - loss: 0.0438 - accuracy: 0.9845 - val_loss: 0.4007 - val_accuracy: 0.9143
Epoch 8/10
1688/1688 [==============================] - 8s 5ms/step - loss: 0.0329 - accuracy: 0.9881 - val_loss: 0.4085 - val_accuracy: 0.9168
Epoch 9/10
1688/1688 [==============================] - 8s 5ms/step - loss: 0.0265 - accuracy: 0.9904 - val_loss: 0.4461 - val_accuracy: 0.9163
Epoch 10/10
1688/1688 [==============================] - 8s 5ms/step - loss: 0.0225 - accuracy: 0.9919 - val_loss: 0.4892 - val_accuracy

In [44]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 26, 26, 48)        480       
                                                                 
 conv2d_1 (Conv2D)           (None, 24, 24, 32)        13856     
                                                                 
 flatten (Flatten)           (None, 18432)             0         
                                                                 
 dense (Dense)               (None, 112)               2064496   
                                                                 
 dense_1 (Dense)             (None, 10)                1130      
                                                                 
Total params: 2,079,962
Trainable params: 2,079,962
Non-trainable params: 0
_________________________________________________________________
